# Software Development Assignment 3: geo-python notebook

In our final project, our team would like to create a Python package for GEE to calculate and map vegetation indices and derive time series. This task aims to obtain the Python API and calculate one of the indices. The Folium package will be used for this purpose. The work will be done in JupyterLab.

## Work beforehand in the anaconda terminal
- move to the directory: ```cd "OneDrive - Universität Salzburg\SS24\Software Development\A3"```
- create a3_gee conda environment:  ```conda create -n a3_gee```
- activate environment: ```conda activate a3_gee```
- installing the ee package: ```conda install -c conda-forge earthengine-api```
- installing pykernel: ```conda install ipykernel```
- installing folium: ```python -m pip install folium```
- installing jupyterlab: ```pip install jupyterlab```
- going in jupyterlab: ```jupyter lab```
- creating a new ipynb file

## Work in Jupyter lab
- importing of ee and folium, and initialization
- setting the area of interest
- setting the time period
- getting the imagery with less cloud
- calculation of indice
- mapping the layer

## Import and initialization

In [1]:
# import ee and folium
import ee
import folium

In [2]:
#authentificate, generate token and allow access rights
ee.Authenticate()

Enter verification code:  4/1AdLIrYe5vL8unfksqXR056gZRJbIBsT9LUpP5rsB5L6zj2bX0pAF2jafDfQ



Successfully saved authorization token.


In [4]:
ee.Initialize()

## Setting the area of interest

In [5]:
# area of interest in olomouc, czechia
aoi = ee.Geometry.Rectangle(17.22,
                            49.61,
                            17.26,
                            49.63)

## Setting period of time

In [6]:
# set the start and end date
start_date = '2023-01-01'
end_date = '2023-12-31'

## Data

In [7]:
# select sentinel 2 imagery
s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(aoi).filterDate(start_date, end_date)
s2_collection = s2_collection.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20);

# amount of images in the selection
print(s2_collection.size().getInfo())

# reduce collection to mean
mean_collection = s2_collection.reduce(ee.Reducer.mean())

C:\Users\gerno\miniconda3\envs\a3_gee\Lib\site-packages\ee\deprecation.py:202: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


48


## Calculating EVI

In [8]:
# function to calculate the EVI
def calculate_evi(image):
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('B8_mean'),
            'RED': image.select('B4_mean'),
            'BLUE': image.select('B2_mean')
        })
    return evi.rename('EVI')

In [9]:
# calculate the evi
evi_mean = calculate_evi(mean_collection)
# clip it to the aoi
evi_mean = evi_mean.clip(aoi)

## Mapping

In [11]:
# function to add the layer to folium map
def add_ee_layer(self, ee_object, color, name):
    map_id_dict = ee.Image(ee_object).getMapId(color)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data GEE",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# add the earth engine function to the folium map
folium.Map.add_ee_layer = add_ee_layer

# create folium map
map = folium.Map(location=[49.61, 17.26], zoom_start = 12)

# setting the vis variables
evi_color = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green']
}

# add the evi layer to the map
map.add_ee_layer(evi_mean, evi_color, 'NDVI')

# display the map
display(map)